## Model 2

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import os

### Step 1 Merge feature data and price data

In [2]:
# Paths to the input files
feature_file_path = 'Data assignment 1/Feature data.csv'
prices_file_path = 'Data assignment 1/prices_merged_df_output.csv'

# Read the CSV files into pandas dataframes
feature_data_df = pd.read_csv(feature_file_path)
prices_data_df = pd.read_csv(prices_file_path)

# Feature scaling, same as model 1
scaler_standard = StandardScaler()

feature_data_df['Mean wind speed'] = scaler_standard.fit_transform(feature_data_df[['Mean wind speed']])
feature_data_df['Maximum temperature'] = scaler_standard.fit_transform(feature_data_df[['Maximum temperature']])

feature_data_df['Wind direction sin'] = np.sin(np.deg2rad(feature_data_df['Mean wind direction']))
feature_data_df['Wind direction cos'] = np.cos(np.deg2rad(feature_data_df['Mean wind direction']))

nominal_capacity = 30000
feature_data_df['AKI Kalby Active Power'] = feature_data_df['AKI Kalby Active Power'] / nominal_capacity

feature_data_df = feature_data_df.drop('Mean wind direction', axis=1)

feature_data_df['datetime'] = pd.to_datetime(feature_data_df['datetime'])
prices_data_df['HourDK'] = pd.to_datetime(prices_data_df['HourDK'])

# Merge the two datasets on their datetime columns, using an inner join to keep only matching rows
merged_df = pd.merge(feature_data_df, prices_data_df, left_on='datetime', right_on='HourDK', how='inner')

# Drop the redundant 'HourDK' column from the merged dataframe
merged_df = merged_df.drop(columns=['HourDK'])

# Define the output directory and file name
output_dir = 'Data assignment 1'
output_path = os.path.join(output_dir, 'Model2data.csv')

# Save the merged dataframe to a CSV file
merged_df.to_csv(output_path, index=False)


### Step 2

#### Regression(based on what we get from model1)

We just used non-linear regression with 4-degree polynomial and ridge regulazation.